In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import AFQ.registration as reg
import AFQ.utils.models as ut
import AFQ.data as afd
import AFQ.tractography as aft
import AFQ.csd as csd
import AFQ.dti as dti
import dipy.data as dpd
import nibabel as nib

In [ ]:
img, gtab = dpd.read_stanford_hardi()

In [ ]:
afq_templates = afd.read_templates()

In [ ]:
afq_templates

In [ ]:
MNI_T2 = dpd.read_mni_template()
MNI_T2_data = MNI_T2.get_data()
MNI_T2_affine = MNI_T2.get_affine()
dwi = img
dwi_affine = dwi.get_affine()
dwi_data = dwi.get_data()
mean_b0 = np.mean(dwi_data[..., gtab.b0s_mask], -1)
warped_b0, mapping = reg.syn_registration(mean_b0, MNI_T2_data,
                                          moving_affine=dwi_affine,
                                          static_affine=MNI_T2_affine,
                                          step_length=0.1,
                                          sigma_diff=2.0,
                                          metric='CC',
                                          dim=3,
                                          level_iters=[10, 10, 5],
                                          prealign=None)



In [ ]:
warped_b0.shape

In [ ]:
mean_b0.shape

In [ ]:
CST_L1 = afq_templates['CST_roi1_L']
CST_L2 = afq_templates['CST_roi2_L']

In [ ]:
CST_L1_data = CST_L1.get_data()
CST_L2_data = CST_L2.get_data()

In [ ]:
warped_CST_L1 = mapping.transform_inverse(CST_L1_data, interpolation='nearest')

In [ ]:
from dipy.segment.mask import median_otsu

In [ ]:
masked_b0, brain_mask = median_otsu(mean_b0)

In [ ]:
plt.matshow(brain_mask[:, :, 40])

In [ ]:
nib.save(nib.Nifti1Image(brain_mask.astype(int), img.affine), 'brain_mask')

In [ ]:
#dti.fit_dti(['/Users/arokem/.dipy/stanford_hardi/HARDI150.nii.gz'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bval'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bvec'], 
#            out_dir='.', mask='brain_mask.nii')

In [ ]:
#nib.save(nib.Nifti1Image((nib.load('dti_FA.nii.gz').get_data() > 0.2).astype(int), img.affine), 'wm_mask.nii.gz')

In [ ]:
#csd.fit_csd(['/Users/arokem/.dipy/stanford_hardi/HARDI150.nii.gz'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bval'], 
#            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bvec'], 
#            out_dir='.',
#            mask='brain_mask.nii')

In [ ]:
mask = nib.load('wm_mask.nii.gz').get_data()

In [ ]:
plt.matshow(mask[:, :, 40])

In [ ]:
csd_coeff = nib.load('csd_sh_coeff.nii.gz').get_data()

In [ ]:
plt.matshow(csd_coeff[:,:,40, 0])

In [ ]:
#localtracking = aft.track('./csd_sh_coeff.nii.gz', seed_density=[1, 1, 1],
#                          seed_mask=nib.load('wm_mask.nii.gz').get_data(), 
#                          stop_mask=nib.load('wm_mask.nii.gz').get_data())

In [ ]:
#streamlines = list(localtracking)

In [ ]:
#sl_long = [sl for sl in streamlines if sl.shape[0]>3]

In [1]:
from AFQ.utils.streamlines import write_trk, read_trk

ImportError: No module named 'AFQ.utils.streamlines'

In [ ]:
write_trk('./csd_streamlines.trk', sl_long)

In [ ]:
## For the corticospinal tract, need to potentially deal with fibers that loop through the brainstem

## Deal with fiber outliers

## Compare with the track probability maps? 